In [1]:
from win32com.client import Dispatch
import time
import numpy as np
import pulsar_dimmer

In [2]:
maxim = Dispatch("MaxIm.Application")
maxim.LockApp = True

In [3]:
camera = Dispatch("MaxIm.CCDCamera")
camera.DisableAutoShutdown = True
camera.LinkEnabled = True
print('Camera connected: %s' % camera.CameraName); print()

readouts = camera.ReadoutModes
for i in range(len(readouts)):
    print('Mode %d: %s' % (i, readouts[i]))
print()
filter_names = camera.FilterNames
for i in range(len(filter_names)):
    print('Filter %d: %s' % (i, filter_names[i]))

Camera connected: DL Imaging

Mode 0: HDR
Mode 1: High Gain
Mode 2: Low Gain
Mode 3: High Gain StackPro

Filter 0: L
Filter 1: 6
Filter 2: V
Filter 3: B
Filter 4: H
Filter 5: W
Filter 6: O
Filter 7: 1
Filter 8: I
Filter 9: X
Filter 10: G
Filter 11: R


In [ ]:
mount = Dispatch('SiTech.Telescope')
print('Homing')
mount.FindHome
mount.Tracking = False
print('Home found')

print('Slewing')
mount.SlewToAltAz(86.96717, 30.09397)
print('Slew complete')
mount.Tracking = False

print('Setting lights')
pulsar_dimmer.dimmer(254)
print('Lights set')

In [4]:
save_loc = 'D:/Calibrations/Darks_2-6-2023/'
readoutLoop = [1, 3]
binLoop = [1]
exposureLoop = [1024]
filLoop = [0]
light = [0]; im_type = {0:'dark', 1:'flat'}
repeat = 12 # number of images per option
estimatedTime = len(readoutLoop)*len(binLoop)*len(filLoop)*len(light)*repeat*sum(exposureLoop)/(60*60)
print('Estimated time to complete: %.2f hours' % estimatedTime)

Estimated time to complete: 6.83 hours


In [5]:
for fil in filLoop:
    for shutter in light:
        for binning in binLoop:
            camera.BinX = binning
            for readout in readoutLoop:
                camera.ReadoutMode = readout
                for exp in exposureLoop:
                    for i in range(repeat):
                        while camera.Temperature > -19:
                            print('Waiting for temperature cooldown...')
                            time.sleep(10)
                        camera.Expose(exp, shutter, fil)
                        if shutter == 0:
                            savestring = save_loc + (im_type[shutter]+'_'+
                                         camera.ReadoutModes[readout].replace(' ', '')+'_'+
                                         str(binning)+'x'+str(binning)+'_'+
                                         # camera.Filternames[fil]+'_'+
                                         str(exp).replace('.', '')+'s__'+
                                         str(i+13)+'.fts')
                        else: 
                            name = 'Flat_%03d%s.fits' % (i+1, camera.Filternames[fil])
                            savestring = save_loc + name
                        while not camera.ImageReady:
                            time.sleep(0.1)
                        print('Saving: %s' % savestring)
                        camera.SaveImage(savestring)

'''print('Turning off lights')
pulsar_dimmer.dimmer(0)
print('Lights off')

print('Homing')
mount.FindHome
mount.Tracking = False
print('Home found')'''

Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__13.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__14.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__15.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__16.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__17.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__18.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__19.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__20.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__21.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__22.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__23.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGain_1x1_1024s__24.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGainStackPro_1x1_1024s__13.fts
Saving: D:/Calibrations/Darks_2-6-2023/dark_HighGainStackPro_1x1_1024

"print('Turning off lights')\npulsar_dimmer.dimmer(0)\nprint('Lights off')\n\nprint('Homing')\nmount.FindHome\nmount.Tracking = False\nprint('Home found')"